In [0]:
spark.conf.set(
    "fs.azure.account.key.imrdatalakedev.dfs.core.windows.net","accessKey")

In [0]:
df = spark.read.format("csv").option("header", "true").load("abfss://bronze@imrdatalakedev.dfs.core.windows.net/Project1_Dividends/CashStatements_*.csv")

In [0]:
df.createOrReplaceTempView("myData")

In [0]:
%sql

SELECT
  COUNT(*)
FROM 
  myData
  WHERE 
  Description LIKE '%DIVIDEND%' 

count(1)
19


In [0]:
%sql
SELECT 
    date_format(to_date(Date, 'dd/MM/yyyy'), 'dd/MMM/yyyy')                     AS PaymentDate,
    --Description                                               AS StockInstrument,
    --`Receipt (GBP)`                   AS AmountPaid
    CAST(`Receipt (GBP)`  AS DECIMAL(10, 2))                                    AS AmountPaid,
    CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(Description, ' ', 2), ' ', -1) AS INT) AS StockUnits,
    INITCAP(regexp_extract(Description, '\\d+\\s+([A-Z\\s]+)\\s+COM', 1) )      AS CompanyName
FROM
  myData
WHERE 
  Description LIKE '%DIVIDEND%' 


PaymentDate,AmountPaid,StockUnits,CompanyName
13/Sep/2024,22.90,115,Realty Income Corp
01/Sep/2024,21.55,227,Intelcorp
15/Aug/2024,23.31,115,Realty Income Corp
15/Jul/2024,23.26,115,Realty Income Corp
20/Jun/2024,22.13,33,Qualcomm Inc
14/Jun/2024,23.78,115,Realty Income Corp
01/Jun/2024,22.20,227,Intelcorp
15/May/2024,23.35,115,Realty Income Corp
15/Apr/2024,23.70,115,Realty Income Corp
21/Mar/2024,20.91,33,Qualcomm Inc


In [0]:
_sqldf \
    .write \
    .format("delta") \
        .save("abfss://silver@imrdatalakedev.dfs.core.windows.net/Project1_Dividends/DividendStatement")